In [1]:
import serial
import time
import struct
import math

In [2]:
# the configuration of the serial port
portName = "COM3"
BAUD_RATES = 115200 # Unit in bit/s

ser = serial.Serial(port = portName,
                    baudrate = BAUD_RATES,
                    bytesize = serial.EIGHTBITS,
                    parity = serial.PARITY_NONE,
                    stopbits = serial.STOPBITS_ONE,
                    timeout = 5) # Unit in second
# ser.flushInput() 

In [3]:
def read_acc():
    readbyte  = 12
    data_bin = ser.read(readbyte)
    fmt = "3f"
    data_decode = struct.unpack_from(fmt, data_bin)

    logVal = abs(math.log(abs(data_decode[0]), 10)) +\
             abs(math.log(abs(data_decode[1]), 10)) +\
             abs(math.log(abs(data_decode[2]), 10))

    if (logVal < 6) and (data_decode[2] > 980.0):
        return data_decode

In [12]:
now = time.strftime("%Y%m%d%H%M%S")
queryFid = "acc3_{}.txt".format(now)

timeout = 1 # Unit in second

start_time = time.time()
sample_num = 0
while time.time() < start_time + timeout:
    try:
        acc_data = read_acc()        
        if acc_data != None:
            with open(queryFid, "a") as file:
                for item in acc_data:
                    file.write("%s," % item)
                file.write("\n")
                sample_num += 1
    except:        
        pass
print("Number of theoretical sample: {} ".format((BAUD_RATES/8/4)*timeout))
print("Number of real sample: {} ".format(sample_num * 3)) # In 3 axis

Number of theoretical sample: 3600.0 
Number of real sample: 1242 


In [103]:
ser.close()